<a href="https://colab.research.google.com/github/2017130744/-neowizard/blob/main/21%20deep%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np

training_data = np.loadtxt('mnist_train.csv', delimiter = ',', dtype = np.float32)
test_data = np.loadtxt('mnist_test.csv', delimiter = ',', dtype = np.float32)

training_data.shape

(60000, 785)

In [7]:
import numpy as np

##1. external function(sigmoid, nemirecal_derivative)

def sigmoid(x):
  return 1 / (1+np.exp(-x))


def numerical_derivative(f,x):
  delta_x = 1e-4
  grad = np.zeros_like(x)        

  it = np.nditer(x, flags=['multi_index'], op_flags = ['readwrite'])

  while not it.finished:
    idx = it.multi_index   

    tmp_val = x[idx]
    x[idx] = float(tmp_val) + delta_x
    fx1 = f(x)    ##f(x + delta_x)

    x[idx] = tmp_val - delta_x
    fx2 = f(x)   ##f(x-delta_x)

    grad[idx] = (fx1 - fx2) / (2*delta_x)

    x[idx] = tmp_val
    it.iternext()

  return grad

In [61]:
class NeuralNetwork:

  def __init__(self, input_nodes, hidden_nodes, output_nodes):

    self.input_nodes = input_nodes
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes

    # 2층 hidden layer unit : 가중치 W2, 바이어스 b2 초기화
    self.W2 = np.random.rand(self.input_nodes, self.hidden_nodes)     # W2 = 784x100
    self.b2 = np.random.rand(self.hidden_nodes)                       # b2 = (100,)

    # 3층 hidden layer unit 
    self.W3 = np.random.rand(self.hidden_nodes, self.output_nodes)     # W3 = 100 x 100
    self.b3 = np.random.rand(self.output_nodes)                         # b3 = (10,)

    #학습률 learning data 초기화
    self.learning_rate = 1e-4



    #feed forward 를 통하여 입력층에서부터 출력층까지 데이터를 전달하고 손실함수 값 계산
    #loss_val(self) 메서드와 동일한 코드. loss_val은 외부 출력용으로 사용됨
  def feed_forward(self):

     delta = 1e-7   ## log 무한대 발산 방지

     z1 = np.dot(self.input_data, self.W2) + self.b2    ##은닉층의 선형회귀 값 계산
     y1 = sigmoid(z1)                                    ##은닉층의 출력

     z2 = np.dot(y1, self.W3) + self.b3              ##출력층의 선형회귀 값
     y = sigmoid(z2)                                ##출력층의 출력

     #cross-entropy
     return -np.sum(self.target_data*np.log(y+delta) + (1-self.target_data)*np.log((1-y)+delta))

    #외부 출력을위한 손실함수 값 계산
  def loss_val(self):

     delta = 1e-7   ## log 무한대 발산 방지

     z1 = np.dot(self.input_data, self.W2) + self.b2    ##은닉층의 선형회귀 값 계산
     y1 = sigmoid(z1)                                    ##은닉층의 출력

     z2 = np.dot(y1, self.W3) + self.b3              ##출력층의 선형회귀 값
     y = sigmoid(z2)                                ##출력층의 출력

     #cross-entropy
     return -np.sum(self.target_data*np.log(y+delta) + (1-self.target_data)*np.log((1-y)+delta))


    #수치미분을 이용하여 손실함수가 최소가 될 때까지 학습하는 함수
  def train(self, training_data):

      #normalize
      self.target_data = np.zeros(output_nodes) + 0.01
      self.target_data[int(training_data[0])] = 0.99

      self.input_data = (training_data[1:] / 255.0 * 0.99) + 0.01

      f = lambda x : self.feed_forward()

      self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
      self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
      self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
      self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)    ##은닉층마다 그 사이에 있는 가중치와 바이어스를 모두 추가해주어야 한다. 


      
    #미래 값 예측 함수
  def predict (self, input_data):
    
     z1 = np.dot(input_data, self.W2) + self.b2           ##은닉층의 선형회귀 값 계산
     y1 = sigmoid(z1)                                    ##은닉층의 출력

     z2 = np.dot(y1, self.W3) + self.b3              ##출력층의 선형회귀 값
     y =  sigmoid(z2)                                ##출력층의 출력

     predicted_num = np.argmax(y)

     return predicted_num

    #정확도 측정함수
  def accuracy(self, test_data):

     matched_list = []
     not_matched_list = []

     for index in range(len(test_data)):

       label = int(test_data[index, 0])

       #normalize
       data = (test_data[index, 1:] / 255.0 * 0.99) + 0.01

       predicted_num = self.predict(data)

       if label == predicted_num:               #정답과 예측값이 일치하면 matched에 넣고 일치하지 않으면 not_matched에 넣는다. 
         matched_list.append(index)
       else:
         not_matched_list.append(index)

     print ('Current Accuracy = ', 100* (len(matched_list) / (len(test_data))), " %")

  

In [ ]:
##mnist 검증

input_nodes = 784
hidden_nodes = 100
output_nodes = 10

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes)

for step in range(30001):  #전체 트레이닝 데이터의 50%
  
  # 총 6만개의 트레이닝 데이터 중 랜덤하게 3만개 선택
  index = np.random.randint(0, len(training_data) - 1)

  nn.train(training_data[index])

  if step % 400 == 0:
    print('step = ', step, ', loss_val = ', nn.loss_val())

In [ ]:
# accuracy 계산

nn.accuracy(test_data)